### Maximization of the Log-Likelihood of Hidden Markov Models on the Limit Order Book

In [9]:
import pandas as pd
import numpy as np
import hmmlearn as hmlearn
from sklearn.model_selection import TimeSeriesSplit

In [10]:
from hmmlearn.hmm import GaussianHMM

In [11]:
features=pd.read_csv('data/features.csv',index_col=0,nrows=1000)
features

,spread,OB_IB
2020-01-02 09:30:00.134062,0.20,1.0
2020-01-02 09:30:00.134336,0.18,1.0
2020-01-02 09:30:00.134532,0.26,2.0
2020-01-02 09:30:00.136081,0.19,1.0
2020-01-02 09:30:00.234474,0.28,1.0
...,...,...
2020-01-02 09:30:28.784959,0.10,2.0
2020-01-02 09:30:28.819508,0.14,2.0
2020-01-02 09:30:28.832168,1.52,1.0
2020-01-02 09:30:29.169166,0.10,1.0


In [12]:
outcomes=pd.read_csv('data/outcomes.csv',index_col=0,nrows=1000)
outcomes

,spread_state,book_state
2020-01-02 09:30:00.134062,0,0
2020-01-02 09:30:00.134336,-1,1
2020-01-02 09:30:00.134532,1,2
2020-01-02 09:30:00.136081,-1,-2
2020-01-02 09:30:00.234474,1,1
...,...,...
2020-01-02 09:30:28.784959,0,0
2020-01-02 09:30:28.819508,1,1
2020-01-02 09:30:28.832168,1,-1
2020-01-02 09:30:29.169166,-1,2


### Time Series Split

In [19]:
tscv = TimeSeriesSplit(n_splits=2)


counter=0
for train_index, test_index in tscv.split(features):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = outcomes.iloc[train_index], outcomes.iloc[test_index]
    

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

### PSG

-Discrete Distributions or Normal Distributions of Observations

In [14]:
import os
os.add_dll_directory('C:\Aorda\PSG\lib')
import psgpython as psg 

### HMM Learn

- Multinomial, Gaussian, Gaussian Mixture States
- Thinking will do Gaussian and transform features to continuous values
- Easier for optimziation and comparison between libraies 

In [22]:
tscv = TimeSeriesSplit(n_splits=5)


counter=0
for train_index, test_index in tscv.split(features):
    
    X_train, X_test = features.iloc[train_index].values, features.iloc[test_index].values
    y_train, y_test = outcomes['spread_state'].iloc[train_index].values, outcomes['spread_state'].iloc[test_index].values
    print(y_train)
    spread_states=3

    spread_model=GaussianHMM(n_components=spread_states,covariance_type='diag',startprob_prior=1.0, transmat_prior=1.0, algorithm='viterbi',params='stmc', init_params='stmc')
    spread_model.startprob_ = X_train[0]
    fitted_spread=spread_model.fit(X_train,y_train)
    log_prob=fitted_spread.score(X_test,y_test)
    print(f"Out of Sample Log Probability is {log_prob}")
    break

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


[ 0 -1  1 -1  1  1  0  1 -1 -1  0  1  1 -1 -1  1  1  1 -1  1 -1  1  0  0
  1 -1  0  0  1 -1 -1  1  1 -1 -1  1 -1  1 -1  0  0  0 -1  1  1 -1  1 -1
 -1  1  1  1 -1  1 -1  0 -1  1 -1  0  0  0  0  0  0  0  1 -1  1 -1 -1  1
  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1 -1  0  0  0  1 -1  0  1 -1  0  0  0  0  0  0  1 -1  0  1 -1
 -1  1  1  0  0 -1  0  0  0  1  0 -1  1 -1  0  1 -1  0  0  1 -1  1  1 -1
  1  1 -1  1 -1  1 -1  0  1 -1  1 -1  0  1 -1  1  1  0 -1  1  0  0 -1  1
  0  1]


C:\Users\jbohn\AppData\Local\Temp\ipykernel_11220\2129338783.py:14: DeprecationWarning: less that 170 samples in lengths array [ 0 -1  1 -1  1  1  0  1 -1 -1  0  1  1 -1 -1  1  1  1 -1  1 -1  1  0  0
  1 -1  0  0  1 -1 -1  1  1 -1 -1  1 -1  1 -1  0  0  0 -1  1  1 -1  1 -1
 -1  1  1  1 -1  1 -1  0 -1  1 -1  0  0  0  0  0  0  0  1 -1  1 -1 -1  1
  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1 -1  0  0  0  1 -1  0  1 -1  0  0  0  0  0  0  1 -1  0  1 -1
 -1  1  1  0  0 -1  0  0  0  1  0 -1  1 -1  0  1 -1  0  0  1 -1  1  1 -1
  1  1 -1  1 -1  1 -1  0  1 -1  1 -1  0  1 -1  1  1  0 -1  1  0  0 -1  1
  0  1]; support for silently dropping samples is deprecated and will be removed
  fitted_spread=spread_model.fit(X_train,y_train)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
book_states=4

book_model=MultinomialHMM(n_components=book_states,startprob_prior=1.0, transmat_prior=1.0, algorithm='viterbi',params='ste', init_params='ste')
book_model.fit()